<a href="https://colab.research.google.com/github/chellabeatrixkiddo/ML_and_DL/blob/master/CV_AnalyticsVidhya/male_vs_female_classification/male_vs_female_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('/content/drive/My Drive/DLDatasets/CV101/datasets/male_vs_female_classification')

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from skimage.transform import resize
from keras.models import Sequential
from keras.layers import Dense, InputLayer

%matplotlib inline

In [0]:

traindata = pd.read_csv("/content/drive/My Drive/DLDatasets/CV101/datasets/male_vs_female_classification/train.csv")

testdata = pd.read_csv("/content/drive/My Drive/DLDatasets/CV101/datasets/male_vs_female_classification/test.csv")

X_train = []
X_test = []

for img_name in traindata.image_names:
    # read the image from location
    img = plt.imread('/content/drive/My Drive/DLDatasets/CV101/datasets/male_vs_female_classification/images/' + img_name)
    # pile it one over the other
    X_train.append(img)

for img_name in testdata.image_names:
    # read the image from location
    img = plt.imread('/content/drive/My Drive/DLDatasets/CV101/datasets/male_vs_female_classification/images/' + img_name)
    # pile it one over the other
    X_test.append(img)

X_train = np.array(X_train)
X_test = np.array(X_test)

y_train = traindata['class']
y_test = testdata['class']

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
  

In [0]:
X_train = preprocess_input(X_train, mode='tf')
X_test = preprocess_input(X_test, mode='tf')

In [0]:
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
base_model.summary()

In [0]:
base_model_pred = base_model.predict(X_train)
base_model_pred_test = base_model.predict(X_test)

In [0]:
base_model_pred = base_model_pred.reshape(12196, 7*7*512)

base_model_pred_test = base_model_pred_test.reshape(5227, 7*7*512)


In [0]:
max_val = base_model_pred.max()

base_model_pred = base_model_pred / max_val
base_model_pred_test = base_model_pred_test / max_val

In [0]:
model = Sequential()
model.add(InputLayer((7*7*512, )))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [0]:
model.compile(optimizer='sgd', metrics=['accuracy'], loss='categorical_crossentropy')
model.summary()

In [0]:
model.fit(base_model_pred, y_train, epochs=15)


In [0]:
predictions = model.predict_classes(base_model_pred_test)
predictions

In [0]:
prediction_probabilities = model.predict(base_model_pred_test)
prediction_probabilities